In [1]:
!pip install azure-ai-textanalytics

    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [2]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
import random
import string
from azure.search.documents import SearchClient
from azure.search.documents.models import IndexAction
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI, OpenAI

In [4]:
#Text Analytics
endpoint = 'https://dev-mbchatbot-lang.cognitiveservices.azure.com/'
key = '2c870c4d7a1343aa8d4921e8f60e8ed1'
text_analytics_client = TextAnalyticsClient(endpoint, AzureKeyCredential(key))

#AI Search
endpoint = 'https://maribelajarsearch.search.windows.net'
api_key = 'ORN9c2qM0ydEwpEO067ZB1SfJS76b7wIitN3aTWHtsAzSeCnCy2N'
index_name = 'maribelajar-faq'

#Azure Open AI
client = AzureOpenAI(
  api_key = '1e923ba14261479ca8cdf3e5fdf857a2',  
  api_version = "2024-08-01",
  azure_endpoint = 'https://maribelajarchatbot.openai.azure.com/'
)

In [5]:
def get_rag_chain(query_text):
    
    search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(api_key))
    search_results = search_client.search(
        search_text=query_text,
        top=5  # Number of documents to retrieve
    )

    docs = []
    docs.append(query_text)
    result = text_analytics_client.analyze_sentiment(docs, show_opinion_mining=True)
    docs = [doc for doc in result if not doc.is_error]
    
    for idx, doc in enumerate(docs):
        sentiment = doc.sentiment

    if (sentiment == "negative") :
        tone = "berempati dan mendukung"
    elif(sentiment == "positive"):
        tone = "mendorong dan positif"
    else:
        tone = "netral dan informatif";
    print(tone)

    messages = [
        {
            "role": "system",
            "content": "Anda adalah asisten AI yang membantu orang menemukan informasi tentang layanan MariBelajar. Berikan sentiment yang sesuai terlebih dahulu seperti sambutan atau respon terhadap sentiment dulu, setelah itu baru jawab pertanyaan berdasarkan informasi yang tersedia"
        },
        {"role": "user", "content": f'''Input Pengguna:{query_text}.
         Informasi dari FAQ: {search_results} .
         Sentimen Pengguna: {sentiment}.
         Buat respons yang menjawab input pengguna menggunakan informasi dari 
         FAQ. Pastikan untuk menggunakan nada {tone} untuk mencerminkan sentimen 
         pengguna."'''}
    ]
    client = AzureOpenAI(api_key ='1e923ba14261479ca8cdf3e5fdf857a2', api_version = "2024-08-01-preview",azure_endpoint = 'https://maribelajarchatbot.openai.azure.com/'
)
    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages,
        temperature=0.7,
        top_p=0.95,
        max_tokens=800,
        stream=False,
        frequency_penalty=0,
        presence_penalty=0

    )
    content = response.choices[0].message.content

    return content

In [6]:
get_rag_chain("Aku lagi sedih tapi lagi ingin ikut webinar, gimana ya caranya?")

berempati dan mendukung


'Halo, saya sangat memahami perasaan sedih Anda. Namun, saya senang mendengar bahwa Anda ingin tetap mengikuti webinar kami di MariBelajar. Untuk mendaftar ke webinar, Anda dapat mengunjungi halaman kami yang berisi daftar webinar yang tersedia dan memilih webinar yang Anda inginkan. Kemudian, klik tombol "Daftar" dan ikuti langkah-langkah pendaftaran. Jangan ragu untuk menghubungi kami jika Anda memerlukan bantuan lebih lanjut. Semoga webinar kami dapat membantu Anda merasa lebih baik!'

In [52]:
get_rag_chain("Haloo aku lagi bersemangat dan ingin ikut webinar di Mari Belajar , gimana ya caranya?")

mendorong dan positif


'Halo! Terima kasih sudah bersemangat untuk mengikuti webinar di Mari Belajar. Untuk mendaftar, kamu bisa mengunjungi halaman acara di website kami dan mengisi formulir pendaftaran yang tersedia. Jangan khawatir, proses pendaftaran sangat mudah dan cepat! Jika ada pertanyaan lebih lanjut, jangan ragu untuk menghubungi tim kami. Selamat belajar!'

In [53]:
get_rag_chain("Cara Daftar Webinar Mari Belajar")

netral dan informatif


'Tentu, untuk mendaftar webinar MariBelajar, silakan mengikuti langkah-langkah berikut:\n1. Kunjungi halaman utama MariBelajar di https://maribelajar.id/\n2. Pilih menu "Webinar" di bagian atas halaman.\n3. Cari webinar yang ingin diikuti dan klik tombol "Daftar".\n4. Isi formulir pendaftaran yang tersedia, dan pastikan untuk mengisi data yang valid.\n5. Setelah selesai mengisi formulir, klik tombol "Kirim".\n6. Anda akan menerima email konfirmasi pendaftaran dari MariBelajar.\n\nSemoga informasi ini membantu. Jangan ragu untuk menghubungi kami jika Anda membutuhkan bantuan lebih lanjut. Terima kasih.'